# Importing libraries

In [1]:
# Importing needed libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Turning project folder into string path 
path = r'C:\Users\keirr\11-2020 Instacart Basket Analysis'

In [3]:
# Importing orders_products_combined.pkl dataset with path
ords_prods_new = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_new_cols.pkl'))

In [4]:
# Checking imported df
ords_prods_new.head(5)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_days,slowest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,NaN,NaN,Regularly busy
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,NaN,Slowest days,Regularly busy
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,NaN,Slowest days,Regularly busy
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,NaN,Slowest days,Least busy
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,NaN,Slowest days,Least busy


# Contents
## 1. Grouping & aggregrating df
## 2. Creating 'loyalty_flag' column
## 3. Checking basic stats of loyalty flags
## 4. Creating 'spending_flag' column
## 5. Creating 'frequency_flag' column
## 6. Exporting df

# 1. Grouping & aggregrating df

In [5]:
# Grouping data by 'department_id', and finding mean of 'order_num'
ords_prods_new.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### Insights: without subsetting the df to include the first 1 million entries, I can see that certain departments were excluded from the data, as well as slightly changing the mean of most 'department_id'

In [6]:
# Grouping data by 'department_id', additionally looking into min and max
ords_prods_new.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.457838   1  99
2                17.277920   1  99
3                17.170395   1  99
4                17.811403   1  99
5                15.215751   1  99
6                16.439806   1  99
7                17.225802   1  99
8                15.340650   1  99
9                15.895474   1  99
10               20.197148   1  99
11               16.170638   1  99
12               15.887671   1  99
13               16.583536   1  99
14               16.773669   1  99
15               16.165037   1  99
16               17.665606   1  99
17               15.694469   1  99
18               19.310397   1  99
19               17.177343   1  99
20               16.473447   1  99
21               22.902379   1  99

# 2. Creating loyalty flag

## Using transform() function to create 'max_order'

In [7]:
# Splitting the data into groups based on the “user_id” column
# Applying the transform() function on the 'order_number' column to generate the max orders for each user
# Creating a new column, 'max_order' for results of agg()

ords_prods_new['max_order'] = ords_prods_new.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\keirr\AppData\Local\Temp\ipykernel_28816\3825906525.py:5: FutureWarning: The provided callable <function amax at 0x000002332A9B1BC0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_new['max_order'] = ords_prods_new.groupby(['user_id'])['order_number'].transform(np.max)


In [9]:
# Checking df
ords_prods_new.head(5)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_days,slowest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,NaN,NaN,Regularly busy,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,NaN,Slowest days,Regularly busy,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,NaN,Slowest days,Regularly busy,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,NaN,Slowest days,Least busy,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,NaN,Slowest days,Least busy,10


## Deriving new column with loc() function

In [10]:
ords_prods_new.loc[ords_prods_new['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

C:\Users\keirr\AppData\Local\Temp\ipykernel_28816\2903401466.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Loyal customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_new.loc[ords_prods_new['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [11]:
ords_prods_new.loc[(ords_prods_new['max_order'] <= 40) & (ords_prods_new['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [12]:
ords_prods_new.loc[ords_prods_new['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
# Checking work
ords_prods_new[['user_id', 'loyalty_flag', 'order_number']].head(15)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


# 3. Checking basic stats of loyalty flags

In [14]:
# Grouping data by 'loyalty flag' & agg 'prices' col

ords_prods_new.groupby('loyalty_flag').agg({'prices':['mean', 'max', 'min']})

prices              
                       mean      max  min
loyalty_flag                             
Loyal customer    10.386336  99999.0  1.0
New customer      13.294670  99999.0  1.0
Regular customer  12.495717  99999.0  1.0

### Insights: 'new customers' have the highest average price per product bought, followed by 'regular customers' and then 'loyal customers'. Perhaps 'new customers' are making infrequent large purchases, whereas 'regular' and 'loyal customers' are making frequent smaller purchases in comparison.

# 4. Creating 'spending_flag' column

## Creating 'mean_price' col w/ transform()

In [15]:
# Splitting the data into groups based on the “user_id” column
# Applying the transform() function on the 'prices' column to generate average price of product for each user
# Creating a new column, 'mean_price' for results of agg()

ords_prods_new['mean_price'] = ords_prods_new.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\keirr\AppData\Local\Temp\ipykernel_28816\2489093551.py:5: FutureWarning: The provided callable <function mean at 0x000002332A9B28E0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_new['mean_price'] = ords_prods_new.groupby(['user_id'])['prices'].transform(np.mean)


In [16]:
# Checking df
ords_prods_new.head(5)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,_merge,price_range_loc,busiest_days,slowest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,7,9.0,both,Mid-range product,NaN,NaN,Regularly busy,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797


## Deriving new variable w/ loc() function

In [17]:
ords_prods_new.loc[ords_prods_new['mean_price'] < 10, 'spending_flag'] = 'Low spender'

C:\Users\keirr\AppData\Local\Temp\ipykernel_28816\956784933.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Low spender' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_new.loc[ords_prods_new['mean_price'] < 10, 'spending_flag'] = 'Low spender'


In [18]:
ords_prods_new.loc[ords_prods_new['mean_price'] >= 10, 'spending_flag'] = "High spender"

In [19]:
# Checking df
ords_prods_new.head(5)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,_merge,price_range_loc,busiest_days,slowest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,both,Mid-range product,NaN,NaN,Regularly busy,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,both,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,both,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,both,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,both,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797,Low spender


# 5. Creating 'frequency_flag' column

## Creating 'median_last_order' column w/ transform()

In [20]:
# Splitting the data into groups based on the 'user_id' column
# Applying the transform() function on the 'days_since_last_order' column to generate frequency of user's shopping
# Creating a new column, 'median_last_order' for results of agg()

ords_prods_new['median_last_order'] = ords_prods_new.groupby(['user_id'])['days_since_last_order'].transform(np.median)

C:\Users\keirr\AppData\Local\Temp\ipykernel_28816\937066586.py:5: FutureWarning: The provided callable <function median at 0x000002332AAF3880> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_new['median_last_order'] = ords_prods_new.groupby(['user_id'])['days_since_last_order'].transform(np.median)


In [21]:
# Checking df
ords_prods_new.head(5)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest_days,slowest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_last_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,both,Mid-range product,NaN,NaN,Regularly busy,10,New customer,6.367797,Low spender,20.5
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797,Low spender,20.5
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797,Low spender,20.5
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797,Low spender,20.5
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797,Low spender,20.5


## Deriving new variable w/ loc() function

In [22]:
ords_prods_new.loc[ords_prods_new['median_last_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

C:\Users\keirr\AppData\Local\Temp\ipykernel_28816\3566390129.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Non-frequent customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_new.loc[ords_prods_new['median_last_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'


In [23]:
ords_prods_new.loc[(ords_prods_new['median_last_order'] > 10) & (ords_prods_new['median_last_order'] <= 20), 'frequency_flag'] = 'Regular customer'

In [24]:
ords_prods_new.loc[ords_prods_new['median_last_order'] <= 10, 'frequency_flag'] = 'Frequenct customer'

In [25]:
# Checking df
ords_prods_new.head(5)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_days,slowest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_last_order,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,NaN,NaN,Regularly busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


# 6. Exporting df

In [26]:
# Exporting grouped and aggregated data to pkl

ords_prods_new.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_agg.pkl'))